Die Analyse bedingt im Kontext dieser Veranstaltung vornehmlich drei Aspekte:
1. Vorverarbeitung der Daten (Bereinigung; Generieren neuer Features, die z.B. aus der Vorgeschichte eines Spiels abgeleitet werden),
2. Analyse von Zusammenhängen(und ggf. Überprüfung der Aussagekraft mit statistischen oder anderen Methoden) und
3. Visualisierung der Erkenntnisse über die ermittelten Zusammenhänge sowie weiterer „interessanter“ Inhalte aus den Daten (ggf. zusammengefasst als einzelne „Data Stories“).

Im Vorfeld wurden bereits konkrete Hypothesen und Fragen formuliert, die Sie überprüfen sollen:
1. Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?
2. Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?
3. Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?
4. Wie entwickelt sich der sportliche Erfolg mit dem Alter? Gilt das für die Spielstatistiken gleichermaßen wie für die Platzierung auf der Rangliste?
5. Sinkt die Wahrscheinlichkeit zu siegen, wenn das vorhergehende Spiel eines Spielers besonders lange gedauert hat?
6. Gibt es einen Zusammenhang dazwischen, wie man zuletzt gegen einen bestimmten Gegner gespielt hat, und wie das nächste Spiel gegen diesen Gegner ausgehen wird?
7. Wie gut lässt sich das Ergebnis eines Matches vorhersagen? Welches sind die wichtigsten Features für eine solche Prognose?

In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tabulate import tabulate

In [113]:
matches = pd.read_csv('../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../data/raw/atp_rankings_till_2022.csv')

## H 1: Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?

1. Identifizierung der Matches, in denen deutsche Spieler beteiligt waren.
2. Ermittlung der Gewinn- und Verlustraten deutscher Spieler gegen Spieler anderer Nationen.
3. Analyse, ob es bestimmte Nationen gibt, gegen die deutsche Spieler signifikant besser oder schlechter abschneiden.

In [114]:
german_ioc_code = "GER"
german_winner_matches = matches[matches["winner_ioc"] == german_ioc_code]
german_loser_matches = matches[matches["loser_ioc"] == german_ioc_code]

siege = german_winner_matches["loser_ioc"].value_counts()
niederlagen = german_loser_matches["winner_ioc"].value_counts()

performance = pd.DataFrame({"Siege": siege, "Niederlagen": niederlagen}).fillna(0)

performance["Total"] = performance.sum(axis=1)
performance["Win_Rate"] = performance["Siege"] / performance["Total"]
performance["Loss_Rate"] = performance["Niederlagen"] / performance["Total"]
performance = performance[performance["Total"] >= 10].sort_values(by="Win_Rate", ascending=False)

top = 10
top_favoriten = performance.head(top)
top_rivalen = performance.tail(top)

In [115]:
performance.sort_values('Win_Rate', ascending=False)

,Siege,Niederlagen,Total,Win_Rate,Loss_Rate
UNK,16.0,1.0,17.0,0.941176,0.058824
VEN,20.0,3.0,23.0,0.869565,0.130435
BOL,12.0,3.0,15.0,0.800000,0.200000
TPE,24.0,7.0,31.0,0.774194,0.225806
PHI,13.0,4.0,17.0,0.764706,0.235294
...,...,...,...,...,...
ZIM,18.0,30.0,48.0,0.375000,0.625000
MAR,25.0,43.0,68.0,0.367647,0.632353
LTU,7.0,15.0,22.0,0.318182,0.681818
UZB,11.0,31.0,42.0,0.261905,0.738095


In [116]:
performance.sort_values('Win_Rate', ascending=False).count().values[0]/2

36.0

In [117]:
# plot win_loss_rate, sorted by win_rate using plotly.express
fig = px.bar(performance.sort_values('Win_Rate', ascending=False), 
            y=['Win_Rate', 'Loss_Rate'], 
            title='Gewinn- und Verlustrate der deutschen Tennisspieler gegen andere Nationen',
            hover_data=['Siege', 'Niederlagen'],
            color_discrete_sequence=["#b2e061", "#fd7f6f"])

fig.add_hline(y=0.5, opacity=0.4, line_dash='dot', line_color='black')

fig.add_annotation(text="50% Gewinnrate",
                xref='paper',
                y=performance.sort_values('Win_Rate', ascending=False).median(),
                arrowhead=1, showarrow=True, arrowcolor='black',        
                bgcolor='white',
                font=dict(size=10, color='grey'))


fig.update_yaxes(tickformat=".2%")
fig.update_yaxes(title='Gewinnrate')
fig.update_xaxes(title='Gegner Nationen')
fig.show()


In [118]:
rankings_unique = rankings.drop_duplicates('player', keep='last')
dfconcat = pd.merge(players, rankings_unique, left_on='player_id', right_on='player', how='left')

## H2: Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?

1. Laden der Daten und Vorbereitung:
- Laden der Match-Daten und Spielerdaten.
- Filtern der Daten für deutsche Spieler.
2. Berechnung der allgemeinen Spielstärke:
- Berechnung der Gewinn-Verlust-Bilanz für jeden deutschen Spieler.
3. Analyse der Leistung gegen spezifische Gegner:
- Extraktion der Gegner für die ersten fünf deutschen Spieler.
- Berechnung der Gewinn- und Verlustbilanz gegen diese Gegner.
4. Identifikation von "Angstgegnern" und "Lieblingsgegnern":
- Vergleich der spezifischen Gewinnrate mit der allgemeinen Gewinnrate.

In [152]:
matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188156,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,4,209362,NaN,NaN,...,68.0,42.0,12.0,10.0,11.0,13.0,1103.0,9.0,1130.0,8.0
188157,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,5,202065,NaN,NaN,...,56.0,40.0,20.0,15.0,4.0,8.0,808.0,23.0,1390.0,4.0
188158,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,1,138846,NaN,NaN,...,54.0,29.0,8.0,11.0,6.0,10.0,1059.0,10.0,1881.0,1.0
188159,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,2,209409,NaN,NaN,...,39.0,24.0,7.0,10.0,5.0,9.0,NaN,NaN,NaN,NaN


In [145]:
matches['l_ace'].sort_values(ascending=True).unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  51.,  52.,  53.,  55.,  56.,
        59.,  61.,  67., 103.,  nan])

In [147]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [153]:
# i only need those columns from matches : 'tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'match_num', 'winner_id', 'winner_hand', 'winner_ioc', 'winner_age', 'looser_id', 'looser_hand', 'looser_ioc', 'looser_age'
matches_filtered = matches[['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'match_num', 'winner_id', 'winner_hand', 'winner_ioc', 'winner_age', 'loser_id', 'loser_hand', 'loser_ioc', 'loser_age', 'winner_rank', 'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points']]

In [154]:
matches_filtered.to_csv('atp_matches_filtered.csv', index=False)

In [144]:
px.histogram(matches, x='w_1stIn', nbins=200)